In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

/home/pallab_workstation/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
part_rad = 1.0
nn_cell_width = 4*part_rad
sq_cutoff_rad = (nn_cell_width)**2

In [3]:
global theta, rad, vel, part_type,  N_WALL, N_PASSIVE, N_ACTIVE,\
            INT_PARTICLES, N_TOTAL, PASSIVE_HALF, x_low, y_low, x_max, y_max, Nc_x, Nc_y, N_cells,\
            x_low_nn, y_low_nn, x_max_nn, y_max_nn, Nc_x_nn, Nc_y_nn, N_cells_nn
part = pd.read_csv("ini_state.txt", header = None, delimiter="\t", names=["x", "y",\
                "radius", "vel","theta","part_type"])#Read values and assigned under these headers
data_pass_0 = np.array(part[part['part_type']==-1][['x','y']])
pos_x = np.asarray(part['x']) #Converts the values under x to an array named x
pos_y = np.asarray(part['y']) #Converts the values under y to an array named y
rad = np.asarray(part['radius']).reshape(-1,1)
part_type = np.asarray(part['part_type']).reshape(-1,1) #Type of particle : 1-interior , 0-boundary

x_low_nn = np.amin(pos_x)-nn_cell_width
y_low_nn = np.amin(pos_y)-nn_cell_width
x_max_nn = np.amax(pos_x)+nn_cell_width
y_max_nn = np.amax(pos_y)+nn_cell_width
Nc_x_nn = int(np.ceil((x_max_nn-x_low_nn)/nn_cell_width))
Nc_y_nn = int(np.ceil((y_max_nn-y_low_nn)/nn_cell_width))
N_cells_nn = int(Nc_x_nn*Nc_y_nn) # Total no of cells

N_WALL = len((part_type[part_type==0]))
N_PASSIVE = len((part_type[part_type==-1]))
N_ACTIVE = len((part_type[part_type==1]))
INT_PARTICLES = N_ACTIVE + N_PASSIVE
N_TOTAL = INT_PARTICLES + N_WALL
PASSIVE_HALF = int(N_PASSIVE/2)
del part # Memory freed

FileNotFoundError: [Errno 2] No such file or directory: 'ini_state.txt'

In [ ]:
#Mixing index based on neighbours
def nn_fn(data_passive):
    
    nn_list = np.zeros(N_PASSIVE,dtype=int)
    posx_nn = data_passive[:,0]
    posy_nn = data_passive[:,1]
    head_nn = np.ones(N_cells_nn)*(-1)
    cell_x_nn = (np.divide(np.subtract(posx_nn,x_low_nn),nn_cell_width)).astype(int) # Putting agent into respective xcell
    cell_y_nn = (np.divide(np.subtract(posy_nn,y_low_nn),nn_cell_width)).astype(int); # Putting agent into respective ycell
    cell_pos_nn = (cell_x_nn*Nc_y_nn + cell_y_nn).astype(int); # Cell number calculated vertically
    
    #assigning particles' (identity) in head2 & nn_list for easily accessing
    for i in range (N_PASSIVE):
        nn_list[i] = head_nn[cell_pos_nn[i]]
        head_nn[cell_pos_nn[i]] = i
        
    pass_type = np.zeros((N_PASSIVE,2))

    for i in range (Nc_x_nn):
        for j in range (Nc_y_nn):
            nn_cell_n = i*Nc_y_nn + j
            iat = int(head_nn[nn_cell_n])
            while (iat != -1): # -1 implies no particles in this head/cell 
                for neigh_cell_x in [i-1,i,i+1]:
                    for neigh_cell_y in [j-1,j,j+1]:
                        neigh_cell_n = neigh_cell_x*Nc_y_nn + neigh_cell_y
                        jat = int(head_nn[neigh_cell_n])
                        while (jat != -1): # -1 implies no particles in this head/cell 
                            if(iat != jat):
                                # Distance calculation of neighboring particles
                                dx = posx_nn[jat] - posx_nn[iat]
                                dy = posy_nn[jat] - posy_nn[iat]
                                sq_distance = dx*dx + dy*dy
                                
                                if (sq_distance <=sq_cutoff_rad):
                                    if(jat<PASSIVE_HALF):
                                        pass_type[iat][0] += 1 # TYPE 1 PASSIVE
                                    if(jat>=PASSIVE_HALF):
                                        pass_type[iat][1] += 1 # TYPE 2 PASSIVE

                            jat = int(nn_list[jat])

                iat = nn_list[iat]
    #print(np.shape(pass_type))
    ratio_1 = np.sum(2*np.nan_to_num(np.divide(pass_type[:PASSIVE_HALF,1],(pass_type[:PASSIVE_HALF,0]+pass_type[:PASSIVE_HALF,1]))),axis=0)
    ratio_2 = np.sum(2*np.nan_to_num(np.divide(pass_type[-PASSIVE_HALF:,0],(pass_type[-PASSIVE_HALF:,0]+pass_type[-PASSIVE_HALF:,1]))),axis=0)
    mix_index = (ratio_1+ratio_2)/N_PASSIVE
#     print(mix_index)
    return mix_index

In [ ]:
tau = [5,10,20,30,40]
Case = np.arange(51,100)
Time = np.arange(10,50001,10)

In [ ]:
ini_time = 10.0
final_time = 50000.0
time_step = 10.0
count = int((final_time-ini_time)/time_step+2) #to contain MI from 0 to 50k in steps of timestep
cases = 50
time_data = np.arange(0,50001,10).reshape(-1,1)

In [ ]:
MI_0 = nn_fn(data_pass_0)
for tr in tau:
    MI = np.zeros((count,cases))
    MI[0,:] = MI_0
    col = 0
    for case in Case:
        row=1
        for time in Time:
            file = f"tau_{tr:d}/Case_{case}/agent_data/t_{time:.2f}.txt"
            df = pd.read_csv(file, delimiter='\t',names=['x','y','theta','counter','part'])
            data_passive = np.array(df[df['part']==-1][['x','y']])
            MI[row][col] = nn_fn(data_passive)
            row+=1
        col+=1
        if (case+1)%10==0: print(f"{case} cases in tau={tr:d} are over")
    np.savetxt(f"tau_{tr:d}/MI_vs_Time_2.csv",np.hstack((time_data,MI)), delimiter='\t')
    print(f"Saved file to tau_{tr:d}")